In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import dash
from dash import dcc, html, Input, Output
import dash_bootstrap_components as dbc
from jupyter_dash import JupyterDash

# Step 1: Data Loading and Preparation
data = pd.read_csv("punjab-crime-stats-with-police.csv")
data['Year'] = pd.to_datetime(data['Year'], format='%Y', errors='coerce')
# Step 2: Initialize the Dash app
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.MINTY])

# Step 3: Define the layout
app.layout = dbc.Container([
    html.Div([
        html.Img(src="/assets/img.png", height="60px"),  # Add your image file path here
        html.H2("Crime Data Analysis Dashboard", style={"margin-left": "10px", "display": "inline-block"})
    ]),
    html.Hr(),
    dbc.Tabs([
        dbc.Tab(label="Overview", tab_id="overview"),
        dbc.Tab(label="Area wise Analysis", tab_id="area-wise-analysis"),
        dbc.Tab(label="Specific Crime Analysis", tab_id="specific-crime-analysis"),
        dbc.Tab(label="About", tab_id="about"),
    ], id="tabs", active_tab="overview"),
    html.Div(id="tab-content"),
])

# Step 4: Define callbacks to switch between tabs
@app.callback(
    Output("tab-content", "children"),
    [Input("tabs", "active_tab")]
)
def render_tab_content(active_tab):
    if active_tab == "overview":
        return html.Div([
            html.H3("Overview"),
            html.P("The Crime Data Analysis Dashboard is a comprehensive tool designed to provide insightful analysis and visualization of crime-related data for the Punjab region."),
            html.P("Built using Python libraries such as Dash, Plotly Express, and pandas, this interactive dashboard offers a user-friendly interface to explore various aspects of crime statistics and trends over time."),
            html.P("Key features of the dashboard include:"),
            html.Ul([
                html.Li("Data Loading and Preparation"),
                html.Li("Interactive Tabs for easy navigation"),
                html.Li("Specific tabs for Area-Wise Analysis and Specific Crime Analysis"),
                html.Li("About tab providing background information"),
                html.Li("Interactive Visualizations using Plotly Express"),
                html.Li("Dropdown Menus for user input"),
                html.Li("User Benefits such as data exploration, decision support, and visual storytelling")
            ])
        ])

    elif active_tab == "area-wise-analysis":
        return html.Div([
            dbc.Row([
                dbc.Col([
                    dcc.Dropdown(
                        id='division-dropdown',
                        options=[{'label': division, 'value': division} for division in data['Division'].unique()],
                        value=data['Division'].unique()[0],
                        multi=False,
                        placeholder="Select Division"
                    ),
                    dcc.Dropdown(
                        id='district-dropdown',
                        options=[],
                        value=[],
                        multi=True,
                        placeholder="Select District(s)"
                    ),
                    html.Label('Year Range'),
                    dcc.RangeSlider(
                        id='year-range-slider',
                        min=data['Year'].dt.year.min(),
                        max=data['Year'].dt.year.max(),
                        value=[data['Year'].dt.year.min(), data['Year'].dt.year.max()],  # Set initial range
                        marks={str(year): str(year) for year in data['Year'].dt.year.unique()},
                        vertical=True,
                        verticalHeight=300,
                        step=None
                    ),
                    html.Div(id='output-container-range-slider'),
                    html.Hr()
                ], md=3),  # Left sidebar
                dbc.Col([
                    dcc.Graph(id='combined-chart'),
                    dcc.Graph(id='crime-yearly-chart'),
                    dcc.Graph(id='crime-population-chart'),
                    dcc.Graph(id='crime-police-chart')
                ], md=9)  # Main area
            ])
        ])
    elif active_tab == "specific-crime-analysis":
        return html.Div([
            html.H3("Specific Crime Analysis"),
            dcc.Dropdown(
                id='crime-dropdown',
                options=[{'label': crime, 'value': crime} for crime in data['CrimeType'].unique()],
                value=data['CrimeType'].unique()[0],
                multi=False,
                placeholder="Select Crime Type"
            ),
            dcc.Graph(id='specific-crime-chart')
        ])
    elif active_tab == "about":
        return html.Div([
            html.H3("About Me"),
            html.Div([
                html.P("Muhammad Hamid Saeed"),
                html.P("Senior Engineer"),
                html.P("The Islamia University of Bahawalpur")
            ], style={"margin-bottom": "20px"}),
            html.H3("Course Details"),
            html.Div([
                html.P("I developed this dashboard during the course:" + '"Certificate in Artificial Intelligence"'+ "in the Departemnt of Electronic Engineering"),
                html.P("Offered in collaboration with PAK-Vision AI group USA"),
                html.P("Supervised by Dr. Abbass Abbasi")
            ], style={"margin-bottom": "20px"}),
            html.P("This dashboard is a culmination of my learning and expertise in data analysis and visualization, as well as my passion for leveraging technology to solve real-world problems."),
            html.P("Data source: Punjab Crime Statistics")
        ])
    else:
        return html.Div("Page not found")
@app.callback(
    Output('district-dropdown', 'options'),
    [Input('division-dropdown', 'value')]
)
def update_district_dropdown(selected_division):
    districts = data[data['Division'] == selected_division]['District'].unique()
    return [{'label': district, 'value': district} for district in districts]
    
# Step 5: Define callback function to update charts based on selected division
@app.callback(
    [Output('combined-chart', 'figure'),
     Output('crime-yearly-chart', 'figure'),
     Output('crime-population-chart', 'figure'),
     Output('crime-police-chart', 'figure')],
    [Input('division-dropdown', 'value'),
     Input('district-dropdown', 'value'),
     Input('year-range-slider', 'value')]
)
def update_charts(selected_division, selected_districts, selected_years):
    filtered_data = data[data['Division'] == selected_division]
    #filtered_data = filtered_data[filtered_data['District'].isin(selected_districts)]
    if selected_districts:
        filtered_data = filtered_data[filtered_data['District'].isin(selected_districts)]
    filtered_data = filtered_data[(filtered_data['Year'].dt.year >= selected_years[0]) & (filtered_data['Year'].dt.year <= selected_years[1])]
    
    # Crimes per year chart
    # crimes_per_year = filtered_data.groupby('Year')['CrimeCount'].sum().reset_index()
    # crime_yearly_chart = px.bar(crimes_per_year, x='Year', y='CrimeCount', title="Crimes per Year")
    crime_rate_yearly = filtered_data.groupby(['Year', 'CrimeType'])['CrimeCount'].sum().reset_index()
    crime_yearly_chart = px.bar(crime_rate_yearly, x='Year', y='CrimeCount', color='CrimeType',
                 title=f"Crime Rate by Crime Type in {selected_districts}")
    crimes_per_year = filtered_data.groupby('Year')['CrimeCount'].sum().reset_index()
    combined_graph=px.bar(crimes_per_year, x='Year', y='CrimeCount', title="Crimes per Year")
    population_count = filtered_data.groupby(['Year','District'])['Population'].unique().reset_index().drop(columns=['District']).groupby('Year').sum().reset_index()
    population_count['Population']=population_count['Population'].apply(lambda x: x[0])
    # Population chart
    population_chart = px.line(population_count, x='Year', y='Population', title="Population over Years")
    
    # Police stations chart (assuming data is available)
    police_stations_count = filtered_data.groupby(['Year','District'])['Police Station'].unique().reset_index().drop(columns=['District']).groupby('Year').sum().reset_index()
    police_stations_count['Police Station']=police_stations_count['Police Station'].apply(lambda x: x[0])
    police_stations_chart = px.line(police_stations_count, x='Year', y='Police Station', title="Police Stations over Years")
    
# Create a bar chart for crime count
# Calculate rate of change for crimes per year
    crimes_per_year['CrimeCount_RateOfChange'] = crimes_per_year['CrimeCount'] / 400

# Calculate rate of change for population count
    population_count['Population_RateOfChange'] = population_count['Population'].diff() / population_count['Year'].diff().dt.days/10

# Calculate rate of change for police stations count
    police_stations_count['PoliceStation_RateOfChange'] = police_stations_count['Police Station'].diff() / police_stations_count['Year'].diff().dt.days

    fig = px.bar(crimes_per_year, x='Year', y='CrimeCount_RateOfChange', title="Crime, Population, and Police Station Counts over Years")
    
    # Add lines for population and police station counts
    fig.add_trace(go.Scatter(x=population_count['Year'], y=population_count['Population_RateOfChange'], mode='lines+markers', name='Population'))
    fig.add_trace(go.Scatter(x=police_stations_count['Year'], y=police_stations_count['Police Station'], mode='lines+markers', name='Police Station'))
    return fig,crime_yearly_chart, population_chart, police_stations_chart
# Define callback function to update text boxes with current min and max values
@app.callback(
    Output('output-container-range-slider', 'children'),
    [Input('year-range-slider', 'value')])
def update_output(value):
    return f'You have selected the year range: {value[0]} - {value[1]}'


# Callback for specific crime analysis
@app.callback(
    Output('specific-crime-chart', 'figure'),
    [Input('crime-dropdown', 'value')]
)
def update_specific_crime_chart(selected_crime):
    filtered_data = data[data['CrimeType'] == selected_crime]
    crime_count_by_year = filtered_data.groupby('Year')['CrimeCount'].sum().reset_index()
    specific_crime_chart = px.line(crime_count_by_year, x='Year', y='CrimeCount', title=f"Trend of {selected_crime}")
    return specific_crime_chart

# Step 6: Run the app
if __name__ == "__main__":
    app.run_server(debug=False)


C:\Users\Hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\dash\dash.py:538: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



Dash app running on http://127.0.0.1:8050/
